## Import Libraries

In [ ]:
import cv2 
import pickle
import matplotlib.pyplot as plt

### Resize Images function

In [ ]:
# Resize images to a similar dimension
# This helps improve accuracy and decreases unnecessarily high number of keypoints

def imageResizeTrain(image):
    maxD = 1024
    height,width = image.shape
    aspectRatio = width/height
    if aspectRatio < 1:
        newSize = (int(maxD*aspectRatio),maxD)
    else:
        newSize = (maxD,int(maxD/aspectRatio))
    image = cv2.resize(image,newSize)
    return image

def imageResizeTest(image):
    maxD = 1024
    height,width,channel = image.shape
    aspectRatio = width/height
    if aspectRatio < 1:
        newSize = (int(maxD*aspectRatio),maxD)
    else:
        newSize = (maxD,int(maxD/aspectRatio))
    image = cv2.resize(image,newSize)
    return image

## Generate Keypoint and Descriptors

### Prepare list of images

In [ ]:
# Define a list of images the way you like

imageList = ["taj1.jpeg","taj2.jpeg","eiffel1.jpeg","eiffel2.jpeg","liberty1.jpeg","liberty2.jpeg","robert1.jpeg","tom1.jpeg","ironman1.jpeg","ironman2.jpeg","ironman3.png","darkknight1.jpeg","darkknight2.jpeg","book1.jpeg","book2.jpeg"]

In [ ]:
# We use grayscale images for generating keypoints
imagesBW = []
for imageName in imageList:
    imagePath = "data/images/" + str(imageName)
    imagesBW.append(imageResizeTrain(cv2.imread(imagePath,0)))

In [ ]:
# # Gray-scasle images improve speed

# imageBW = []
# for image in images:
#     imageBW.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

### Choose between opencv's SIFT and open source SIFT implementation<br>

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

def computeSIFT(image):
    return sift.detectAndCompute(image, None)

In [ ]:
# import pysift

# def computeSIFT(image):
#     return pysift.computeKeypointsAndDescriptors(image)

### The following is the main function to generate the keypoints and descriptors<br>
When using SIFT, this takes a lot of time to compute.<br>
Thus, it is suggested, you store the values once computed<br>
(Code for storing is written below)

In [ ]:
keypoints = []
descriptors = []
i = 0
for image in imagesBW:
    print("Starting for image: " + imageList[i])
    keypointTemp, descriptorTemp = computeSIFT(image)
    keypoints.append(keypointTemp)
    descriptors.append(descriptorTemp)
    print("  Ending for image: " + imageList[i])
    i += 1

### Store Keypoints and Descriptors for future use

In [ ]:
i = 0
for keypoint in keypoints:
    deserializedKeypoints = []
    filepath = "data/keypoints/" + str(imageList[i].split('.')[0]) + ".txt"
    for point in keypoint:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, point.class_id)
        deserializedKeypoints.append(temp)
    with open(filepath, 'wb') as fp:
        pickle.dump(deserializedKeypoints, fp)    
    i += 1

In [ ]:
i = 0
for descriptor in descriptors:
    filepath = "data/descriptors/" + str(imageList[i].split('.')[0]) + ".txt"
    with open(filepath, 'wb') as fp:
        pickle.dump(descriptor, fp)
    i += 1

## Prepare for fetching results

### Fetch Keypoints and Descriptors from stored files

In [ ]:
def fetchKeypointFromFile(i):
    filepath = "data/keypoints/" + str(imageList[i].split('.')[0]) + ".txt"
    keypoint = []
    file = open(filepath,'rb')
    deserializedKeypoints = pickle.load(file)
    file.close()
    for point in deserializedKeypoints:
        temp = cv2.KeyPoint(x=point[0][0],y=point[0][1],_size=point[1], _angle=point[2], _response=point[3], _octave=point[4], _class_id=point[5])
        keypoint.append(temp)
    return keypoint

In [ ]:
def fetchDescriptorFromFile(i):
    filepath = "data/descriptors/" + str(imageList[i].split('.')[0]) + ".txt"
    file = open(filepath,'rb')
    descriptor = pickle.load(file)
    file.close()
    return descriptor

### Calculate Results for any pair

In [ ]:
def calculateResultsFor(i,j):
    keypoint1 = fetchKeypointFromFile(i)
    descriptor1 = fetchDescriptorFromFile(i)
    keypoint2 = fetchKeypointFromFile(j)
    descriptor2 = fetchDescriptorFromFile(j)
    matches = calculateMatches(descriptor1, descriptor2)
    score = calculateScore(len(matches),len(keypoint1),len(keypoint2))
    plot = getPlotFor(i,j,keypoint1,keypoint2,matches)
    print(len(matches),len(keypoint1),len(keypoint2),len(descriptor1),len(descriptor2))
    print(score)
    plt.imshow(plot),plt.show()

In [ ]:
def getPlotFor(i,j,keypoint1,keypoint2,matches):
    image1 = imageResizeTest(cv2.imread("data/images/" + imageList[i]))
    image2 = imageResizeTest(cv2.imread("data/images/" + imageList[j]))
    return getPlot(image1,image2,keypoint1,keypoint2,matches)

### Basic Scoring metric
A score greater than 10 means very good

In [ ]:
def calculateScore(matches,keypoint1,keypoint2):
    return 100 * (matches/min(keypoint1,keypoint2))

### Use this part of code for brute force matching

In [ ]:
# bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
# def calculateMatches(descriptor1,descriptor2):
#     matches = bf.match(descriptor1,descriptor2)
#     matches = sorted(matches, key = lambda x:x.distance)
#     return matches

In [ ]:
# def getPlot(image1,image2,keypoint1,keypoint2,matches):
#     matchPlot = cv2.drawMatches(image1, keypoint1, image2, keypoint2, matches[:50], image2, flags=2)
#     return matchPlot

### Use this part of code for knn matching

In [ ]:
bf = cv2.BFMatcher()
def calculateMatches(des1,des2):
    matches = bf.knnMatch(des1,des2,k=2)
    topResults1 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults1.append([m])
            
    matches = bf.knnMatch(des2,des1,k=2)
    topResults2 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults2.append([m])
    
    topResults = []
    for match1 in topResults1:
        match1QueryIndex = match1[0].queryIdx
        match1TrainIndex = match1[0].trainIdx

        for match2 in topResults2:
            match2QueryIndex = match2[0].queryIdx
            match2TrainIndex = match2[0].trainIdx

            if (match1QueryIndex == match2TrainIndex) and (match1TrainIndex == match2QueryIndex):
                topResults.append(match1)
    return topResults

In [ ]:
# Use this if you want faster results and can give up some accuracy
# bf = cv2.BFMatcher()
# def calculateMatches(des1,des2):
#     matches = bf.knnMatch(des1,des2,k=2)
#     topResults = []
#     for m,n in matches:
#         if m.distance < 0.7*n.distance:
#             topResults.append([m])
#     return topResults

In [ ]:
def getPlot(image1,image2,keypoint1,keypoint2,matches):
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    matchPlot = cv2.drawMatchesKnn(image1,keypoint1,image2,keypoint2,matches,None,[255,255,255],flags=2)
    return matchPlot

## Fetch Results

### Sample Results

In [ ]:
calculateResultsFor(13,14)
